In [3]:
import pandas as pd
import numpy as np
import torch

In [4]:
df = pd.read_csv("watch_train.csv")

In [ ]:
df.columns

## 傾向スコアの計算

In [ ]:
df.price_ave

In [ ]:
df.light_or_dark

In [ ]:

# 'light_or_dark'が'dark'である行をフィルタリング
dark_data = df[df['light_or_dark'] == 1]
# 'prive_ave'が1である行の割合を計算
probability = (dark_data['price_ave'] == 1).mean()

print(f"light_or_darkがdarkであるもののうち、prive_aveが1である確率: {probability}")

In [ ]:
# 'light_or_dark'が'dark'である行をフィルタリング
dark_data = df[df['light_or_dark'] == 0]
# 'prive_ave'が1である行の割合を計算
probability = (dark_data['price_ave'] == 1).mean()

print(f"light_or_darkがlightであるもののうち、prive_aveが1である確率: {probability}")

## 結果のシミュレーション

In [6]:
price_ave_given_dark_probs = np.array([0.29,0.33])

In [33]:
def outcome_sim(beta0, beta1, gamma, treatment, confounding, noise, setting = "simple"):
    if setting == "simple":
        y0 = beta1 * confounding
        y1 = beta0 * y0

        simulated_score = (1. - treatment) * y0 + treatment * y1 + gamma * noise
    return simulated_score, y0,y1
import numpy.random as random
from scipy.special import expit
def make_price_dark_probs(df,treat_strength,con_strength, noise_level, setting = "simple", seed = 0):
    price_ave_given_dark_probs = np.array([0.29,0.33])

    np.random.seed(seed)
    all_noise = np.array(random.normal(0,1,len(df)), dtype = np.float32)
    all_thresholds = np.array(random.uniform(0,1,len(df)), dtype = np.float32)
    
    outcomes = []
    y0s = []
    y1s = []

    for i, data in df.iterrows():
        light_or_dark = 1 if data["light_or_dark"] == "light" else 0
        treatment = data["price_ave"]

        confounding = 3.0 * (price_ave_given_dark_probs[light_or_dark] - 0.25)
        noise = all_noise[i]
        y,y0,y1 = outcome_sim(treat_strength, con_strength, noise_level,treatment, confounding, noise, setting = setting)
        print(y,y0,y1)
        simulated_prob = expit(y)
        y0 = expit(y0)
        y1 = expit(y1)
        threshold = all_thresholds[i]
        simulated_outcome = 1 if simulated_prob > threshold else 0
        outcomes.append(simulated_outcome)
        y0s.append(y0)
        y1s.append(y1)
        
    df['outcome'] = outcomes
    df['y0'] = y0s
    df['y1'] = y1s
    return df

In [ ]:
df = make_price_dark_probs(df,0.5, 5.0,0.0,"simple", 0)

In [ ]:
df.outcome.value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df["y1"],bins=20)

In [38]:
df.to_csv("outputs_v4.csv",index = None)

In [ ]:
df.head()